Bloco 1 - Configuração Inicial e Importação de Bibliotecas
Descrição:
Este bloco monta o Google Drive para permitir acesso aos arquivos armazenados na nuvem durante a execução no Google Colab. Em seguida, importa as bibliotecas essenciais para o projeto:

json para manipulação de arquivos JSON,

datasets para trabalhar com datasets estruturados no formato HuggingFace,

transformers para importar modelos pré-treinados, tokenizadores e utilitários para treinamento.

In [1]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Importar bibliotecas
import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments


Mounted at /content/drive


Bloco 2 - Funções Utilitárias para Carregamento e Filtragem de Dados
Descrição:
Aqui são definidas funções auxiliares fundamentais para o processamento dos dados:

load_labels: carrega um conjunto de labels (rótulos) a partir de um arquivo .txt, eliminando linhas vazias.

stream_json_lines: lê arquivos JSON no formato JSON Lines (um objeto JSON por linha) de forma streaming, para economizar memória.

filtro_por_label: filtra itens JSON verificando se um campo de label (pode ser label, code ou title) pertence a um conjunto de labels pré-definido. Essas funções facilitam o carregamento e filtragem seletiva dos dados que serão usados no treinamento.

In [ ]:
# Função para carregar labels do arquivo txt
def load_labels(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return set(line.strip() for line in f if line.strip())

# Função para ler JSON Lines em streaming
def stream_json_lines(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                yield json.loads(line)

# Função para filtrar item baseado no label
def filtro_por_label(item, label_set):
    label = item.get('label') or item.get('code') or item.get('title')
    return label in label_set


Bloco 3 - Carregamento, Filtragem e Preparação do Dataset para Fine-Tuning
Descrição:
Este bloco realiza a carga e filtragem dos dados principais para o fine-tuning:

Define funções para ler arquivos JSON Lines completos e para carregar labels de filtro que podem conter múltiplos rótulos por linha.

Filtra os datasets de treino e teste usando os labels carregados, garantindo que somente os exemplos relevantes sejam mantidos para treinamento e avaliação.

Constrói exemplos no formato esperado para fine-tuning, onde para cada item filtrado é criado um par input (prompt contendo a pergunta sobre o título) e content (descrição do produto).

Por fim, imprime informações resumidas do volume de dados carregados e filtrados, preparando o dataset para o próximo passo.



In [ ]:
import json

# Função para carregar arquivos JSON Lines
def load_json_lines(filepath):
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                obj = json.loads(line)
                data.append(obj)
    return data

# Função para carregar labels a partir dos arquivos de filtro (múltiplos labels por linha)
def load_labels_multi_per_line(file_path):
    label_set = set()
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            labels = line.strip().split()
            label_set.update(labels)  # adiciona todas as labels da linha
    return label_set

# Função para verificar se qualquer target_ind do item está no label_set
def filtro_por_label(item, label_set):
    target_inds = item.get('target_ind') or []
    target_inds = map(str, target_inds)  # garante que todos são strings
    return any(label in label_set for label in target_inds)

# Carregar labels de filtro
train_labels = load_labels_multi_per_line('/content/drive/MyDrive/Colab Notebooks/Datasets/filter_labels_train.txt')
test_labels = load_labels_multi_per_line('/content/drive/MyDrive/Colab Notebooks/Datasets/filter_labels_test.txt')

print(f"Labels de treino carregadas: {len(train_labels)}")
print(f"Labels de teste carregadas: {len(test_labels)}")

# Carregar datasets
trn_data = load_json_lines('/content/drive/MyDrive/Colab Notebooks/Datasets/trn.json')
tst_data = load_json_lines('/content/drive/MyDrive/Colab Notebooks/Datasets/tst.json')

print(f"Total de registros no dataset de treino: {len(trn_data)}")
print(f"Total de registros no dataset de teste: {len(tst_data)}")

# Filtrar datasets conforme labels
trn_data_filtrado = [item for item in trn_data if filtro_por_label(item, train_labels)]
tst_data_filtrado = [item for item in tst_data if filtro_por_label(item, test_labels)]

print(f"Itens de treino após filtro: {len(trn_data_filtrado)}")
print(f"Itens de teste após filtro: {len(tst_data_filtrado)}")

# Preparar exemplos para fine-tuning
train_examples = []
for item in trn_data_filtrado:
    title = item.get('title')
    content = item.get('content')
    if title and content:
        pergunta = f"Quais as informações sobre o produto '{title}'?"
        input_text = f"Pergunta: {pergunta} Título: {title}"
        target_text = content
        train_examples.append({"input": input_text, "target": target_text})

print(f"Total de exemplos preparados para o treino: {len(train_examples)}")


Bloco 4 - Setup do Ambiente, Tokenização, Configuração e Execução do Fine-Tuning com LoRA
Descrição:
Este bloco:

Instala pacotes necessários (peft, accelerate, transformers, datasets) para fine-tuning eficiente.

Seleciona uma pequena amostra (~3%) do dataset para treinar rapidamente uma versão de teste do modelo.

Prepara o dataset para o HuggingFace Trainer, fazendo tokenização e criação dos inputs/labels para modelo causal de linguagem.

Configura o modelo TinyLlama-1.1B-Chat-v1.0 para fine-tuning usando a técnica LoRA (Low-Rank Adaptation), que reduz custo computacional ajustando menos parâmetros.

Define argumentos de treinamento com parâmetros como batch size, número de passos, taxa de aprendizado, etc., focando em um treinamento rápido e com poucas épocas.

Executa o processo de fine-tuning usando o Trainer da HuggingFace.

Salva o modelo e tokenizer treinados para uso posterior.

In [ ]:
!pip install peft accelerate transformers datasets --quiet

In [ ]:
import random
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import get_peft_model, LoraConfig, TaskType

# Supondo que train_examples já esteja disponível

# Selecionar 3% do dataset para treino rápido
total_examples = len(train_examples)
subset_size = max(1, int(total_examples * 0.03))  # pelo menos 1 exemplo
train_subset = random.sample(train_examples, subset_size)

print(f"Total exemplos para treino: {total_examples}")
print(f"Usando subset de: {subset_size} exemplos para treinamento")

# Criar Dataset HuggingFace
dataset = Dataset.from_list(train_subset)

# Carregar tokenizer e modelo
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token  # evitar erro de padding

max_seq_length = 128  # reduzido para acelerar

def preprocess_function(examples):
    inputs = [f"{inp} {tgt}" for inp, tgt in zip(examples["input"], examples["target"])]
    model_inputs = tokenizer(
        inputs,
        max_length=max_seq_length,
        truncation=True,
        padding="max_length",
    )
    # Labels = input_ids para causal LM
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

tokenized_dataset = dataset.map(
    preprocess_function, batched=True, remove_columns=["input", "target"]
)

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=4,
    lora_alpha=16,
    lora_dropout=0.05,
)

model = get_peft_model(model, lora_config)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir="./tinyllama_lora_finetuned",
    per_device_train_batch_size=4,  # Se sua VRAM aguentar, até 8
    gradient_accumulation_steps=1,
    num_train_epochs=1,             # Uma só época
    max_steps=100,                  # Limita para 100 passos de treino!
    save_strategy="no",
    logging_steps=50,
    learning_rate=5e-4,             # Mais agressivo para convergir rápido
    weight_decay=0.01,
    warmup_steps=10,                # Pouco warmup
    fp16=True,
    report_to="none",
    push_to_hub=False,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

model.save_pretrained("./tinyllama")
tokenizer.save_pretrained("./tinyllama")


Total exemplos para treino: 1354376
Usando subset de: 40631 exemplos para treinamento


Map:   0%|          | 0/40631 [00:00<?, ? examples/s]

<ipython-input-15-ae00e95efb9f>:83: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
50,1.887200
100,1.680200


('./tinyllama/tokenizer_config.json',
 './tinyllama/special_tokens_map.json',
 './tinyllama/tokenizer.model',
 './tinyllama/added_tokens.json',
 './tinyllama/tokenizer.json')

 Hiperparâmetros utilizados na geração de texto
1. max_length=100
Define o máximo total de tokens (entrada + saída).

Exemplo: se entrada = 50 tokens → só restam 50 tokens para a resposta.

Alternativa recomendada: max_new_tokens=50 → controla apenas a quantidade de tokens gerados.

2. do_sample=False
Geração determinística: sempre escolhe o token mais provável → respostas previsíveis.

Método: greedy decoding (sem aleatoriedade).

Alternativa:

do_sample=True → ativa sampling → respostas mais criativas.

3. Outras opções úteis (não usadas mas recomendadas):
temperature=0.7 → mais criatividade (quanto maior, mais variado).

top_p=0.9 → nucleus sampling → considera só os tokens mais prováveis até acumular 90% de chance.

repetition_penalty=1.2 → evita repetições.








Bloco 5 - Comparação entre Modelo Base e Modelo Fine-Tunado
Descrição:
Este bloco é destinado a carregar o modelo base e o modelo fine-tunado para comparação de performance:

Para fins de teste, geramos dois modelos fine-tunados. Um com menos treinamento e outro com mais treinamento, depois que fiz a compra do colab pro conseguimos utilizar maquinas melhores e acelerar o treinamento.

Importa os três modelos (base e fine-tunados) com seus respectivos tokenizadores.

Cria pipelines de geração de texto para ambos.

Monta um exemplo de pergunta usando um título de produto.

Gera respostas com os dois modelos para o mesmo prompt.

Exibe as respostas lado a lado para facilitar a comparação de melhorias no conteúdo gerado após o fine-tuning, destacando ganhos em relevância e precisão.



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Caminhos para os modelos
caminho_modelo_finetunado = "/content/tinyllama"
nome_modelo_base = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Carregar tokenizer
tokenizer = AutoTokenizer.from_pretrained(nome_modelo_base)

# Carregar modelo base
modelo_base = AutoModelForCausalLM.from_pretrained(nome_modelo_base)

# Carregar modelo finetunado
modelo_finetunado = AutoModelForCausalLM.from_pretrained(caminho_modelo_finetunado)

# Criar pipelines
pipeline_base = pipeline("text-generation", model=modelo_base, tokenizer=tokenizer, device=0)
pipeline_finetunado = pipeline("text-generation", model=modelo_finetunado, tokenizer=tokenizer, device=0)

# Gerar respostas
resposta_base_raw = pipeline_base(texto_entrada, max_length=100, do_sample=False)[0]['generated_text']
resposta_finetunada_raw = pipeline_finetunado(texto_entrada, max_length=100, do_sample=False)[0]['generated_text']

# Função para extrair apenas a parte gerada após o input
def extrair_resposta(texto_entrada, resposta_raw):
    return resposta_raw[len(texto_entrada):].strip()

# Extrair respostas limpas
resposta_base = extrair_resposta(texto_entrada, resposta_base_raw)
resposta_finetunada = extrair_resposta(texto_entrada, resposta_finetunada_raw)

##############################COLOQUE O TITULO DO PRODUTO AQUI######################
exemplo_titulo = "Girls Ballet Tutu Neon Pink"
pergunta = f"What are the Information about the product '{exemplo_titulo}'?"
texto_entrada = f"Pergunta: {pergunta}"

# Exibir respostas
print("=== Resposta do Modelo Base ===")
print(resposta_base)
print("\n" + "="*40 + "\n")

print("=== Resposta do Modelo Finetunado ===")
print(resposta_finetunada)
print("\n" + "="*40 + "\n")

print("=== Observações ===")
print("Observe possíveis melhorias em relevância, detalhes ou precisão na resposta do modelo finetunado.")


Device set to use cuda:0
Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


=== Resposta do Modelo Base ===
- Resposta: The Girls Ballet Tutu Neon Pink is a beautiful and elegant tutu dress that is perfect for ballet performances. It features a beautiful neon pink tutu with a delicate lace overlay. The tutu is made of high-quality fabric and is designed to be comfortable and breathable. The tutu is


=== Resposta do Modelo Finetunado ===
Título: Girls Ballet Tutu Neon Pink Girls Ballet Tutu Neon Pink is a beautifully designed tutu with a neon pink tutu and a matching tutu skirt. The tutu is made of a soft, stretchy fabric and has a beautifully designed tutu skirt. The tutu sk


=== Observações ===
Observe possíveis melhorias em relevância, detalhes ou precisão na resposta do modelo finetunado.


Bloco 6 - Interface Interativa para Testes com o Modelo Fine-Tunado

Descrição:
Este último bloco implementa uma interface simples e interativa no terminal para testar o modelo fine-tunado:

Instala bibliotecas necessárias para inferência.

Carrega o tokenizer e modelo fine-tunado previamente salvo.

Cria uma pipeline para geração de texto com o modelo.

Define uma função para gerar respostas para qualquer pergunta recebida.

Em loop, permite ao usuário digitar perguntas sobre produtos em inglês (compatível com dados de teste) e imprime as respostas do modelo.

O usuário pode encerrar o programa digitando "sair".

Essa etapa serve para validar e demonstrar a aplicabilidade prática do modelo treinado.

In [ ]:
# Instalar dependências necessárias
!pip install transformers accelerate peft


In [ ]:

# Importar bibliotecas
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Caminho para o modelo fine-tuned menos treinado
caminho_modelo_afinadado = "/content/drive/MyDrive/Colab Notebooks/tiny/content/tinyllama"

# Caminho para o modelo fine-tuned mais treinado
caminho_modelo_finetunado = "/content/drive/MyDrive/Colab Notebooks/loralhama"

# Nome do modelo base
nome_modelo_base = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Carregar tokenizer e modelo afinadado
tokenizer1 = AutoTokenizer.from_pretrained(caminho_modelo_afinadado)
modelo_afinadado = AutoModelForCausalLM.from_pretrained(caminho_modelo_afinadado)

# Carregar tokenizer e modelo fine-tuned
tokenizer2 = AutoTokenizer.from_pretrained(caminho_modelo_finetunado)
modelo_finetunado = AutoModelForCausalLM.from_pretrained(caminho_modelo_finetunado)

# Criar pipeline 1
pipeline_afinadado = pipeline("text-generation", model=modelo_afinadado, tokenizer=tokenizer1, device=0)


# Criar pipeline 2
pipeline_finetunado = pipeline("text-generation", model=modelo_finetunado, tokenizer=tokenizer2, device=0)

# Função para gerar resposta
def responder1(titulo):
    entrada = f"{titulo}"
    resposta = pipeline_afinadado(entrada, max_length=150, do_sample=True, temperature=0.7)[0]['generated_text']
    return resposta

def responder2(titulo):
    entrada = f"{titulo}"
    resposta = pipeline_finetunado(entrada, max_length=150, do_sample=True, temperature=0.7)[0]['generated_text']
    return resposta

# Interface interativa
print("Modelo carregado com sucesso! Faça perguntas sobre produtos Em ingles para combinar com dados de teste.\n")

while True:
    titulo = input("Digite a pergunta do produto (ou 'sair' para encerrar): ")
    if titulo.lower() == 'sair':
        break
    resposta1 = responder1(titulo)
    resposta2 = responder2(titulo)
    print("\nPergunta Realizada:",titulo,"\n")
    print("\nResposta do modelo 1:\n")
    print(resposta1)
    print("\nResposta do modelo 2:\n")
    print(resposta2)
    print("\n" + "="*50 + "\n")


Device set to use cuda:0
Device set to use cuda:0


Modelo carregado com sucesso! Faça perguntas sobre produtos Em ingles para combinar com dados de teste.

Digite a pergunta do produto (ou 'sair' para encerrar): Whats is the tutu baller 


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Pergunta Realizada: Whats is the tutu baller  


Resposta do modelo 1:

Whats is the tutu baller 1303251673113393-1-1303251673113393-1-1303251673113393-1-1303251673113393-1-1303251673113393-1-1303251673113393-1-1303251673113393-1-1303251

Resposta do modelo 2:

Whats is the tutu baller 101?
Design a tutorial using the latest technology.
Analyze the impact of technology on the creative process.
Design a virtual reality environment.
Develop and test a virtual reality experience.
Design a virtual reality experience for a consumer product.
Create a digital art piece using virtual reality.
Design a virtual reality game for children.
Design a virtual reality headset that can be used in a museum.
Design a virtual reality headset for a medical simulation.
Design a virtual reality experience for a theme park.
Create a virtual reality experience for a cultural event.
Design a virtual reality experience for a museum.


Digite a pergunta do produto (ou 'sair' para encerrar): Tutu Ballet

Pergunta

✅ Considerações Finais do Projeto
Objetivo cumprido: Foi realizado com sucesso o fine-tuning de um foundation model (TinyLlama) para responder perguntas baseadas no título dos produtos da Amazon, utilizando o dataset AmazonTitles-1.3MM.

Processo estruturado: O pipeline completo foi implementado, desde a montagem do ambiente e carregamento dos dados, passando pela preparação, filtragem e formatação dos exemplos para fine-tuning.

Uso de LoRA: A técnica LoRA permitiu realizar fine-tuning eficiente e rápido em uma pequena amostra do dataset, reduzindo o custo computacional sem perda significativa da qualidade do ajuste.

Avaliação comparativa: O modelo fine-tunado foi comparado com a versão base, demonstrando melhorias na geração das respostas, mais alinhadas com as descrições reais dos produtos.

Interface prática: Uma interface interativa simples foi criada para que usuários possam testar o modelo e obter respostas em tempo real, validando sua aplicabilidade.

Próximos passos sugeridos: ampliar o volume de dados para treinar com mais exemplos, ajustar hiperparâmetros para melhorar a qualidade, explorar outras arquiteturas de foundation models, e eventualmente implementar uma API para uso mais robusto do modelo.